In [1]:
import selenium.webdriver as webdriver
from bs4 import BeautifulSoup
from pyvirtualdisplay import Display
from time import sleep
import datetime

import pandas as pd
from tqdm import tqdm

In [2]:
display = Display(visible=0,size=(1024, 768))
display.start()

In [3]:
driver = webdriver.Chrome('/home/ubuntu/chromedriver')

In [4]:
def get_table():
    
    def find_id(element):
        return str(element)[25:33]
    
    def chart(n):
        driver.get(f'https://www.genie.co.kr/chart/top200?pg={n}')
        page = driver.page_source
        soup = BeautifulSoup(page, 'lxml')
        
        table = soup.find(attrs={'class':'newest-list'}) \
                    .find_all('tr', attrs={'class':'list'})
        
        return [find_id(e) for e in table]
        
    return chart(1) + chart(2) + chart(3) + chart(4)

In [5]:
song_ids = get_table()

In [6]:
class song_page():
    
    def __init__(self, id):
        self.id = id
    
    def get_page(self):
        url = f"https://www.genie.co.kr/detail/songInfo?xgnm={self.id}"
        self.url = url

        driver.get(url)
        song_page = driver.page_source
        song_soup = BeautifulSoup(song_page, 'lxml')
        self.soup = song_soup
        
    def get_name(self):
        self.name = self.soup.find('h2',attrs={'class':'name'}).text.strip()

    def get_values(self):
        values = self.soup.find('ul',attrs={'class':'info-data'}) \
                          .find_all(attrs={'class':'value'})
        self.artist    = values[0].text
        self.album     = values[1].text
        self.genre     = values[2].text
        self.playtime  = values[3].text
        #self.lyricists = values[4].text
        #self.composers = values[5].text
        #self.arrangers = values[6].text

    def get_likes(self):
        self.likes = self.soup.find('em',attrs={'id':'emLikeCount'}).text

    def get_totals(self):
        counts = self.soup.find('div', attrs={'class':'total'}) \
                     .find_all('div')
        self.listners = counts[0].text.strip()
        self.numplays = counts[1].text.strip()
        
    def get_comments(self):
        self.comments = self.soup.find('span', attrs={'class':'article'}).text

    def get_now(self):
        self.now = datetime.datetime.now()

    def set_attributes(self):
        self.get_page()

        self.get_name()
        self.get_values()
        self.get_likes()
        self.get_totals()
        self.get_comments()
        self.get_now()

    def get_attributes(self):
        return (self.id, self.name,self.artist,self.album,self.genre,self.playtime,
                 #self.lyricists,self.composers,self.arrangers,
                 self.likes,
                 self.listners,self.numplays,
                 self.comments,
                 self.now)

### 곡 하나에 대해서 실험

In [7]:
first_song = song_page('90705547')
first_song.set_attributes()
first_song.get_attributes()

('90705547',
 'Dynamite',
 '방탄소년단',
 'Dynamite (DayTime Ver.)',
 '가요 / 댄스',
 '03:19',
 '89,413',
 '2,408,868',
 '109,881,819',
 '(총 2489개)',
 datetime.datetime(2021, 7, 8, 4, 1, 45, 419356))

### 모든 곡에 대해 적용

In [ ]:
song_info = []

for sid in tqdm(song_ids):
    song = song_page(sid)
    song.set_attributes()
    song_info.append(song.get_attributes())

 37%|███▋      | 74/200 [01:05<01:51,  1.13it/s]

In [ ]:
Songs = pd.DataFrame(song_info, 
                     columns = ['id',
                                'name','artist','album','genre','playtime',
                                #'lyricists','composer','arranger',
                                'likes',
                                'listners','numplays',
                                'comments',
                                'time(now)'])

In [ ]:
Songs

In [ ]:
Songs.info()

In [ ]:
#driver.quit()
#display.stop()